# GenDivRange Data Preprocessing Documentation

In [ ]:
# load modules and dataframes
import os
import re
import pandas as pd
import numpy as np

spec_df = pd.read_excel(r'spec_tab_***.xlsx')
pop_df = pd.read_excel(r'pop_tab_***.xlsx')

## Geog_1 at level of country
We add a column named Geog_1 to indicate location of a population at the level of country, which is obtained either by country code or information from source paper or reverse geocoding from coordinates.

In [ ]:
# load modules and functions
import requests

def reverse_geocoder(Lat,Lon):
    '''return a string of top 3 matching location
    
    Input Lat and Lon can be string or numeric'''
    API_KEY = 'ENTER_YOUR_API_KEY'
    url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng='+str(Lat)+','+str(Lon)+'&key='+API_KEY
    ini_string = ''
    try:
        response = requests.get(url).json()
        if len(response['results']) <= 3:
            for i in range(len(response['results'])):
                ini_string += response['results'][i]['formatted_address'] + ' '
        else:
            for i in range(3):
                ini_string += response['results'][i]['formatted_address'] + ' '

        return ini_string
    except:
        return 'Error Found'

def find_country(string):
    '''find and return a country within a string of location 
    
    input string should contains an address with country'''
    
    try:
        if 'USA' in string or 'United States' in string:
            return 'United States of America'
        country_list = ['India', 'Uganda', 'Kenya', 'Tanzania', 'South Africa', 'Botswana','Namibia', 'Portugal', 'Netherlands', 'Czechia', 'Russia',
        'Estonia', 'Latvia', 'Spain', 'Germany', 'Sweden', 'Norway', 'Finland', 'Switzerland', 'United States of America',
        'United Kingdom', 'Atlantic Ocean ', 'Atlantic', 'Mexico', 'Malawi', 'Australia', 'Canada', 'Mediterranean Sea',
        'Tyrrhenian Sea', 'Italy', 'Tunisia', 'Algeria', 'Mozambique', 'France', 'Brazil', 'Uruguay', 'Argentina', 'Japan', 'Greece',
        'Romania', 'Turkey', 'Slovenia', 'Greenland ', 'Greenland', 'Greenland Sea', 'North Sea', 'Norwegian Sea', 'New Zealand',
        'New Zealand ', 'Tasman Sea', 'Albania', 'Ghana', 'Zimbabwe', 'Zambia', 'Panama', 'Malaysia', 'Soma', 'Somalia', 'Belgium', 'Denmark', 'Irish Sea', 'Iceland', 'Senegal', 'Mongolia', 'China',
        'Madagascar', 'Indonesia', 'Ireland', 'Ecuador', 'Poland', 'Egypt', 'Thailand', 'Singapore', 'Guatemala', 'Costa Rica',
        'North Pacific Ocean', 'Cambodia', 'Vietnam', 'Peru', 'Colombia', 'Venezuela', 'Taiwan', 'South Korea', 'Caribbean Sea', 'Honduras',
        'The Bahamas', 'United Kindom', 'Belize', 'South Pacific Ocean', 'Austria', 'Iran', 'Isle of Man', 'Eswatini', 'Hungary',
        'Turkmenistan ', 'Kazakhstan', 'Montenegro', 'Bosnia and Herzegovina', 'Baltic Sea', 'Ethiopia',
        'French Polynesia', 'Cook Islands', 'Fiji', 'Papua New Guinea', 'Morocco', 'Israel', 'Jordan', 'Slovakia', 'Bulgaria', 'Ukraine',
        'Moldova', 'Niger', 'Barbados', 'Malta', 'Skagerrak', 'Vanuatu', 'Beaufort Sea', 'Antarctica', 'Weddell Sea', 'Réunion',
        'Philippines', 'Guam', 'New Caledonia', 'Jamaica', 'North Atlantic Ocean', 'Georgia', 'Armenia', 'Alboran Sea',
        'Serbia', 'Trinidad and Tobago', 'Paraguay', 'Myanmar', 'Cyprus', 'Bering Sea', 'Chile', 'American Samoa', 'Tyrrhenian Sea ',
        'East Sea', 'Croatia', 'Beibu Gulf', 'Laos', "Côte d'Ivoire", 'South Sudan', 'Nicaragua', 'Bolivia', 'French Guiana', 'Puerto Rico', 'Cuba', 'Saint Lucia', 'Guyana']
        for i in country_list:
                if i in string:
                    return i
        if 'UK' in string or 'British' in string or 'Cayman Islands' in string or 'Montserrat' in string or 'Bermuda' in string or 'Falkland Islands' in string:
            return 'United Kindom'
        elif 'Dominican Republic' in string or 'Dominica' in string:
            return 'Dominican Republic'
        elif 'Jan Mayen' in string:
            return 'Norway'
        elif 'Guadeloupe' in string:
            return 'France'
    except:
        return 'Error'

In [ ]:
# create a column 'api' to store the address obatined from google map api using coordinates
pop_df['api'] = ''
for i in range(len(pop_df)):
    Lat = pop_df.at[i,'Latitude']
    Lon = pop_df.at[i,'Longitude']
    pop_df.at[i,'api'] = reverse_geocoder(Lat, Lon)
    if i % 1000 == 0:
        print('now finished at ', i, ' with study id: ', pop_df.loc[i,'Study_id'])

# extract key word country using function find_country
for i in range(len(pop_df)):
    pop_df.at[i,'Geog_1'] = find_country(pop_df.at[i,'api'])
    if i % 1000 == 0:
        print('currently at: ', i)

## Fishbase
Key words are extracted from fishbase database in order to determine habitat later

In [ ]:
# load functions and module
import requests
import re

def fishbase(species):
    '''return classfication and environment of the species if found in fishbase.de
    
    input a species (in 'Genus species' format)'''
    def replace_all(text,dict):
        for key,value in dict.items():
            text = text.replace(key,value)
        return text

    dict = {'usually':'', ' depth range ':'', ';c':'', ';n':'', ';s':'', ';e':'', ';w':'', 'temperature':'', ' ph range:':'', ' dh range:':'', '; ,   .':'', ';;':'', '?':'', '..;':'', '  ; ':'',' , ':''}
    try:
        spec = species.replace(' ','-')
        url = "https://www.fishbase.de/summary/" + spec + ".html"
        request = requests.get(url)
        if request.ok:
            s = request.text
            start = "<!-- start Environment / Climate / Range -->"
            end = "<!-- start Distribution -->"
            target = s[s.find(start)+len(start):s.rfind(end)].lower()   # this extracts the Environment partition
            if 'species name is not in the public version of fishbase' in target:
                return 'Not found'
            taxa = re.sub(r'<.*?>' ,'', re.sub(r'[(\r)(\n)(\t)0-9]','',target) ).replace('environment: milieu / climate zone / depth range / distribution rangeecology','').replace('&deg','').replace(' - ','').replace('&nbsp','').replace('ref.','').replace(' m ','')
            result = replace_all(taxa, dict)
            return result
        else:
            return 'Format incorrect'
    except:
        return 'Error'

In [ ]:
spec_df['fishbase'] = ''
for i in range(len(spec_df)):
    spec_df.at[i,'fishbase'] = fishbase(spec_df.at[i,'Species'])
    if i % 1000 == 0:
        print(i)

## EOL dababase
We extract many useful information of each species from EOL database, e.g., the common name, and short overview of the species. Addtionally, we also recorded teh species page url of the EOl database.

In [ ]:
# load modules needed
import os
import requests
import wget
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from time import sleep

In [ ]:
spec_df['found'] = [None] * len(spec_df)
spec_df['common_name'] = [None] * len(spec_df)
spec_df['page_id'] = [None] * len(spec_df)
spec_df['overview'] = [None] * len(spec_df)

In [ ]:
url = 'https://eol.org/'
options = Options()
options.add_argument("user-data-dir=C:\\Users\\yanghao\\AppData\\Local\\Google\\Chrome\\User Data\\Profile 2")
options.add_argument('headless') # not open the browser

for i in range( len(spec_df) ):
    spec_name = spec_df.at[i,'Species']
    driver = webdriver.Chrome('C:\\Users\\yanghao\\Anaconda3\\chromedriver.exe', options=options)
    driver.get(url)
    driver.implicitly_wait(2)
    className = driver.find_element_by_name("q")
    className.send_keys(spec_name + '\n')
    try:
        res = driver.find_element_by_xpath("(//ul[@class='search-results js-search-results']/li)[contains(a,'Creatures')]").click()
        spec_df.at[i,'found'] = True
        # Now at final page of desired info
        spec_df.at[i,'page_id'] = driver.current_url
        
        try:
            spec_df.at[i,'common_name'] = driver.find_element_by_tag_name('h1').text
        except NoSuchElementException:
            spec_df.at[i,'common_name'] = 'not found'

        try:
            spec_df.at[i,'full_latin'] = driver.find_element_by_tag_name('h2').text
        except NoSuchElementException:
            spec_df.at[i,'full_latin'] = 'not found'

        try:
            desc = driver.find_element_by_xpath("(//*[@class='desc']/p)[1]").text
            spec_df.at[i,'overview'] = desc
        except NoSuchElementException:
            spec_df.at[i,'overview'] = 'not found'

    except:
        spec_df.at[i,'found'] = False
    finally:
        sleep(1)
    if i % 20 == 0:
        print('now finished at ', i, ' with species: ', spec_df.loc[i,'Species'])

## Number of loci

## Taxonomy

Taxonomy is extracted from overview text from EOL database. In total, species are categorized into 14 Taxonomy classes (Birds, Fishes, Amphibians, Mammals, Reptiles, Woody plants, Other inverebrates, Algae, Herbaceous plants, Molluscs, Crustaceans, Insects, Mosses, and Fungi).

In [ ]:
spec_df['Taxonomy'] = [None] * len(spec_df)

for i in range(len(spec_df)):
    try:
        spec_df.at[i,'Taxonomy'] = re.search('.*?is a s.*?pecies of (.*?) in the', spec_df.overview[i]).group(1)
    except:
        spec_df.at[i,'Taxonomy'] = 'Not found'
    if i % 200 == 0:
        print('now finished at ', i, ' with species: ', spec_df.loc[i,'Species'])

taxo_dict = {
    'birds':'Birds',
    'bony fishes':'Fishes',
    'amphibians':'Amphibians',
    'mammals':'Mammals',
    'crocodilians':'Reptiles',
    'snakes':'Reptiles',
    'rodents':'Mammals',
    'turtles':'Reptiles',
    'Squamata':'Reptiles',
    'lampreys':'Fishes',
    'tree':'Woody plants',
    'annual herb':'Herbaceous plants',
    'perennial herb':'Herbaceous plants',
    'herb':'Herbaceous plants',
    'shrub':'Woody plants',
    'grass':'Herbaceous plants',
    'annual grass':'Herbaceous plants',
    'woody plants':'Woody plants',
    'echinoderms':'Other inverebrates',
    'dinoflagellates':'Algae',
    'Gastropoda':'Molluscs',
    'decapods':'Crustaceans',
    'comb jellies':'Other inverebrates',
    'mussels':'Molluscs',
    'cnidarians':'Other inverebrates',
    'segmented worms':'Other inverebrates',
    'arrow worms':'Other inverebrates',
    'Lepidoptera':'Insects',
    'beetles':'Insects',
    'Cicadomorpha':'Insects',
    'primates':'Mammals',
    'bats':'Mammals',
    'mosses':'Mosses',
    'flies':'Insects',
    'modern sharks':'Fishes',
    'true bugs':'Insects',
    'rays':'Fishes',
    'Hymenoptera':'Insects',
    'nematodes':'Other inverebrates',
    'Myliobatiformes':'Fishes'   
}

for i in range(len(spec_df)):
    if spec_df.at[i,'Taxonomy'] in taxo_dict.keys():
        spec_df.at[i,'Taxonomy'] = taxo_dict[ spec_df.at[i,'Taxonomy'] ]

 ## BIOME from WWF Terrestrial Ecoregions of the World
 This section fills the column named BIOME that indicates the terrestrial ecoregions for each species.

In [ ]:
# load function and module required
import geopandas as gpd

def get_BIOME(df, LonColIdx, LatColIdx, path = "C:\\Users\\yanghao\\Downloads\\ecology_map\\official\\wwf_terr_ecos.shp"):
    '''
    Get Terrestrial ecosystems and BIOME from WWF based on coordinates. Geopandas required; wwf_terr_ecos.shp required 
    
    Parameters:
        df: input dataframe which include coordinates (assume EPSG:4326)
        LonColIdx: column index of the DD format of Longitude
        LatColIdx: column index of the DD format of Latitude
        path: the path of of file wwf_terr_ecos.shp

    Return:
        The DataFrame with two additional columns indicating detailed ecology name and BIOME
    '''

    # check if input LonColIdx or LatColIdx is int
    if (type(LonColIdx) is not int) | (type(LatColIdx) is not int):
        raise TypeError('input LonColIdx or LatColIdx must be Int')

    wwf = gpd.read_file(path).to_crs('EPSG:4326')
    df2 = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.iloc[:,LonColIdx], df.iloc[:,LatColIdx]))
    df2.crs = 'EPSG:4326'
    df2['wwf_ecology'] = ''
    df2['BIOME_num'] = ''
    df2['BIOME'] = ''

    habitat_dict = {
        1: 'Tropical & Subtropical Moist Broadleaf Forests',
        2: 'Tropical & Subtropical Dry Broadleaf Forests',
        3: 'Tropical & Subtropical Coniferous Forests',
        4: 'Temperate Broadleaf & Mixed Forests',
        5: 'Temperate Conifer Forests',
        6: 'Boreal Forests/Taiga',
        7: 'Tropical & Subtropical Grasslands, Savannas & Shrublands',
        8: 'Temperate Grasslands, Savannas & Shrublands',
        9: 'Flooded Grasslands & Savannas',
        10: 'Montane Grasslands & Shrublands',
        11: 'Tundra',
        12: 'Mediterranean Forests, Woodlands & Scrub',
        13: 'Deserts & Xeric Shrublands',
        14: 'Mangroves',
        98: 'Lake',
        99: 'Rock and Ice',
        100: 'Others'
    }

    for i in range(len(df2)):
            for j in range(len(wwf)):
                if df2.at[i,'geometry'].within(wwf.at[j,'geometry']):
                    df2.at[i,'wwf_ecology'] = wwf.at[j,'ECO_NAME']
                    df2.at[i,'BIOME_num'] = wwf.at[j,'BIOME']
                    df2.at[i,'BIOME'] = habitat_dict[ df2.at[i,'BIOME_num'] ]
                    break
            else:
                df2.at[i,'wwf_ecology'] = 'unknown'
                df2.at[i,'BIOME_num'] = 100
                df2.at[i,'BIOME'] = 'Others'
    
    return df2

# load occurrence data for each species, which is obtained from GBIF DATABASE

occurrence_df = pd.read_csv(r'PATH_TO_OCCURRENCE_DATASET') # Obtained from GBIF database; see R script 
occurrence_df2 = get_BIOME(occurrence_df, LonColIdx, LatColIdx)
summary = occurrence_df2.groupby('Species')['BIOME'].value_counts().unstack().fillna(0)

summary2 = summary.copy()
summary['BIOME'] = ''

for i in list(pd.unique(occurrence_df2['Species'])):
    summary.at[i,'BIOME'] = summary2.loc[i,].idxmax()

summary['Species'] = summary.index
spec_df.merge(summary.loc[:,['Species', 'BIOME']], how = 'left', on = 'Species')

# For these species that have no occurrences data on GBIF, we use populations coordinates to obtain the BIOME

## Habitat_breeding & Habitat_adulthood
Given the speciaty of some fish species, we decided to split the Habitat column into Habitat_breeding & Habitat_adulthood.

In [ ]:
spec_df['Habitat_breeding'] = ''
spec_df['Habitat_adulthood'] = ''

for i in range(len(spec_df)):
    if spec_df.at[i,'Taxonomy'] in ['Birds', 'Reptiles', 'Woody plants','Herbaceous plants','Insects','Mosses','Fungi']:
        spec_df.at[i, 'Habitat_breeding'] = 'Terrestrial'
        spec_df.at[i, 'Habitat_adulthood'] = 'Terrestrial'
    elif spec_df.at[i,'Taxonomy'] == 'Amphibians':
        spec_df.at[i, 'Habitat_breeding'] = 'Freshwater'
        spec_df.at[i, 'Habitat_adulthood'] = 'Terrestrial'
    elif spec_df.at[i,'Taxonomy'] == 'Fishes':
        if 'potamodromous' in spec_df.at[i,'fishbase']:
            spec_df.at[i, 'Habitat_breeding'] = 'Freshwater'
            spec_df.at[i, 'Habitat_adulthood'] = 'Freshwater'
        elif 'anadromous' in spec_df.at[i,'fishbase']:
            spec_df.at[i, 'Habitat_breeding'] = 'Freshwater'
            spec_df.at[i, 'Habitat_adulthood'] = 'Marine'
        elif 'catadromous' in spec_df.at[i,'fishbase']:
            spec_df.at[i, 'Habitat_breeding'] = 'Marine'
            spec_df.at[i, 'Habitat_adulthood'] = 'Freshwater'
        elif 'amphidromous' in spec_df.at[i,'fishbase']:
            spec_df.at[i, 'Habitat_breeding'] = ''
            spec_df.at[i, 'Habitat_adulthood'] = ''
        elif 'oceanodromous' in spec_df.at[i,'fishbase']:
            spec_df.at[i, 'Habitat_breeding'] = 'Marine'
            spec_df.at[i, 'Habitat_adulthood'] = 'Marine'
        elif ('marine' in spec_df.at[i,'fishbase']) & ('freshwater' not in spec_df.at[i,'fishbase']):
            spec_df.at[i, 'Habitat_breeding'] = 'Marine'
            spec_df.at[i, 'Habitat_adulthood'] = 'Marine'
        elif ('freshwater' in spec_df.at[i,'fishbase']) & ('marine' not in spec_df.at[i,'fishbase']):
            spec_df.at[i, 'Habitat_breeding'] = 'Freshwater'
            spec_df.at[i, 'Habitat_adulthood'] = 'Freshwater'

In [ ]:
spec_df.to_excel('species_upload_HabitatNotCompleted.xlsx')